# Introduction to the Reverse Osmosis Unit Model
This tutorial shows how to build, initialize, simulate, and optimize a reverse osmosis (RO) unit model using WaterTAP. In this example, we will minimize the specific energy consumption of the RO unit.

In [1]:
# Python magics from dsl.py
%load_ext dsl
# Utility functions to retrieve result objects
from dsl import get_model, get_results

In [2]:
%%iic
start
# create unit
unit RO-0D
# fix values
set inlet flow mass NaCl 0.035
set inlet flow mass H2O 0.965
set inlet pressure 50e5
set inlet temperature 298.15
set membrane area 50
set membrane permeability water 4.2e-12
set membrane permeability salt 3.5e-8
set permeate pressure 101325
# scale
scale flow mass NaCl 1e2
scale flow mass H2O 1
# solve
init
solve

Running: start
Running: # create unit
Running: unit RO-0D
Running: # fix values
Running: set inlet flow mass NaCl 0.035
Running: set inlet flow mass H2O 0.965
Running: set inlet pressure 50e5
Running: set inlet temperature 298.15
Running: set membrane area 50
Running: set membrane permeability water 4.2e-12
Running: set membrane permeability salt 3.5e-8
Running: set permeate pressure 101325
Running: # scale
Running: scale flow mass NaCl 1e2
Running: scale flow mass H2O 1
Running: # solve
Running: init
2022-02-02 09:09:28 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.area
2022-02-02 09:09:28 [INFO] idaes.init.fs.unit: Initialization Step 1 Complete.
2022-02-02 09:09:29 [INFO] idaes.init.fs.unit: Initialization Step 2 Complete.
2022-02-02 09:09:29 [INFO] idaes.init.fs.unit: Initialization Step 3 successful.
2022-02-02 09:09:29 [INFO] idaes.init.fs.unit: Initialization Complete: optimal - Optimal Solution Found
Running: solve


In [3]:
get_model().fs.unit.display()

Block fs.unit

  Variables:
    A_comp : Solvent permeability coeff.
        Size=1, Index=fs.unit.A_comp_index, Units=m**2*s/kg
        Key          : Lower : Value   : Upper : Fixed : Stale : Domain
        (0.0, 'H2O') : 1e-18 : 4.2e-12 : 1e-06 :  True :  True : NonNegativeReals
    B_comp : Solute permeability coeff.
        Size=1, Index=fs.unit.B_comp_index, Units=m/s
        Key           : Lower : Value   : Upper : Fixed : Stale : Domain
        (0.0, 'NaCl') : 1e-11 : 3.5e-08 : 1e-05 :  True :  True : NonNegativeReals
    area : Membrane area
        Size=1, Index=None, Units=m**2
        Key  : Lower : Value : Upper  : Fixed : Stale : Domain
        None :   0.1 :  50.0 : 1000.0 :  True :  True : NonNegativeReals
    recovery_mass_phase_comp : Mass-based component recovery
        Size=2, Index=fs.unit.recovery_mass_phase_comp_index
        Key                  : Lower : Value                 : Upper    : Fixed : Stale : Domain
         (0.0, 'Liq', 'H2O') :  0.01 :   0.30933

## Stopped here! Code below is standard IDAES Python

## Step 7: Unfix variables, set variable bounds, and run optimization to minimize specific energy consumption.

In [ ]:
# Unfix membrane area and feed pressure
m.fs.unit.area.unfix()                  # membrane area (m^2)
m.fs.unit.inlet.pressure[0].unfix()     # feed pressure (Pa)

In [ ]:
# Set lower and upper bounds for membrane area (m^2)
m.fs.unit.area.setlb(1)
m.fs.unit.area.setub(500)

In [ ]:
# Set lower and upper bounds for feed pressure (Pa)
m.fs.unit.inlet.pressure[0].setlb(10e5)
m.fs.unit.inlet.pressure[0].setub(80e5)

In [ ]:
# Assume 100% efficiency of pumps and ERD and no pressure losses
#--> Pump power consumption ~ Qp*Pf/3.6e6
m.fs.specific_energy_consumption = Expression(
    expr=m.fs.unit.inlet.pressure[0]/(3.6e6))

In [ ]:
# Define objective function to minimize the specific energy consumption.
m.fs.objective = Objective(expr=m.fs.specific_energy_consumption)

In [ ]:
# Set the water recovery to 50%
m.fs.unit.recovery_vol_phase[0,'Liq'].fix(0.50)

In [ ]:
# The solver will find the membrane area and 
# inlet pressure that achieve 50% recovery while minimizing
# specific energy consumption. Since we fixed the 
# volumetric water recovery, a degree of freedom 
# was removed from the model.
print(degrees_of_freedom(m))

In [ ]:
optimization_results = solver.solve(m)

In [ ]:
# membrane area of the optimized RO unit
value(m.fs.unit.area)

In [ ]:
# inlet pressure of the optimized RO unit
value(m.fs.unit.inlet.pressure[0])

In [ ]:
# the minimum specific energy consumption
value(m.fs.specific_energy_consumption)

In [ ]:
# display the overall report on the RO unit
m.fs.unit.report()

In [4]:
class T:
    def __getitem__(self, key):
        print(f"getitem, key={key}")

In [5]:
t = T()

In [6]:
t[1,23]

getitem, key=(1, 23)


In [7]:
x = [1,2,3]

In [8]:
t[x]

getitem, key=[1, 2, 3]


In [11]:
t[tuple(x)]

getitem, key=(1, 2, 3)
